# Build air quality non-attainment suitablity layers for GRIDCERF


The following code was used to build the air quality non-attainment suitability layers for GRIDCERF. GRIDCERF does not provide the source data directly due to some license restrictions related for direct redistribution of the unaltered source data.  However, the following details the provenance associated with each source dataset and how they were processed.


### 1.1 Download GRIDCERF

Download the GRIDCERF package if you have not yet done so from here:  https://doi.org/10.5281/zenodo.6601789.  Please extract GRIDCERF inside the `data` directory of this repository as the paths in this notebook are set to that expectation.


### 1.3 Import Python modules


In [5]:
import os
import glob
import shutil
import tempfile

import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import rasterio.mask
from rasterio import features
import shapely.speedups



## 2. Configuration


In [6]:
# get the parent directory path to where this notebook is currently stored
root_dir = os.path.dirname(os.getcwd())

# data directory in repository
data_dir = os.path.join(root_dir, "data")

# GRIDCERF data directory from downloaded archive
gridcerf_dir = os.path.join(data_dir, "gridcerf")

# GRIDCERF reference data directory
reference_dir = os.path.join(gridcerf_dir, "reference")

# GRIDCERF source data directory for the downloaded airport data
source_dir = os.path.join(gridcerf_dir, "source", "technology_specific", "epa_non-attainment")

# GRIDCERF technology_specific data directory
technology_specific_dir = os.path.join(gridcerf_dir, "technology_specific")

# GRIDCERF compiled final suitability data directory
compiled_dir = os.path.join(gridcerf_dir, "compiled")

# template land mask raster
template_raster = os.path.join(reference_dir, "gridcerf_landmask.tif")

# CONUS boundary shapefile
conus_shpfile = os.path.join(reference_dir, "gridcerf_conus_boundary.shp")

# input shapefiles
co_file = os.path.join(source_dir, "CO_1971std_naa.shp")
no2_file = os.path.join(source_dir, "NO2_1971std_naa.shp")
ozone_file = os.path.join(source_dir, "ozone_8hr_2015std_naa.shp")
lead_file = os.path.join(source_dir, "Pb_NAA_2008.shp")
pm10_file = os.path.join(source_dir, "PM10_1987std_naa.shp")
pm25_file = os.path.join(source_dir, "PM25_2012Std_NAA.shp")
so2_file = os.path.join(source_dir, "so2_2010std_naa.shp")

# output rasters
output_co_file = os.path.join(technology_specific_dir, "gridcerf_epa_nonattainment_co_conus.tif")
output_no2_file = os.path.join(technology_specific_dir, "gridcerf_epa_nonattainment_no2_conus.tif")
output_ozone_file = os.path.join(technology_specific_dir, "gridcerf_epa_nonattainment_ozone_conus.tif")
output_lead_file = os.path.join(technology_specific_dir, "gridcerf_epa_nonattainment_lead_conus.tif")
output_pm10_file = os.path.join(technology_specific_dir, "gridcerf_epa_nonattainment_pm10_conus.tif")
output_pm25_file = os.path.join(technology_specific_dir, "gridcerf_epa_nonattainment_pm2p5_conus.tif")
output_so2_file = os.path.join(technology_specific_dir, "gridcerf_epa_nonattainment_so2_conus.tif")


## 3. Generate non-attainment suitability rasters


### 3.1 Functions to build suitability

In [10]:
def vector_to_raster(template_raster: str, 
                     gdf: gpd.GeoDataFrame, 
                     value_field: str, 
                     output_raster: str,
                     all_touched: bool = True):
    """Convert a vector layer to a raster in the GRIDCERF format.
    
    :param template_raster:         Full path with file name and extension to the input template 
                                    raster using the GRIDCERF format.
    :type template_raster:          str 
    
    :param gdf:                     Geodataframe for the vector data.
    :type gdf:                      gpd.GeoDataFrame
    
    :param value_field:             Field to use to burn raster value in.
    :type value_field:              str
    
    :param output_raster:           Full path with output name and extension for the output raster.
    :type output_raster:            str
    
    :param all_touched:             If True, all pixels touched by geometries will be burned in. 
                                    If false, only pixels whose center is within the polygon or that 
                                    are selected by Bresenham’s line algorithm will be burned in.
    :type all_touched:              bool
    """
                     
    # open the template raster and extract metadata and land mask
    with rasterio.open(template_raster) as template:

        metadata = template.meta.copy()

        # update raster data type
        metadata.update(dtype=np.int16)

        # extract land mask
        land_mask = template.read(1)
        land_mask = np.where(land_mask == 0, np.nan, 1)

        # write output raster
        with rasterio.open(output_raster, 'w+', **metadata) as out:

            out_arr = out.read(1)

            # build shapes to rasterize from target geometry and field
            shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf[value_field]))

            # burn features
            burned = features.rasterize(shapes=shapes, 
                                        fill=0, 
                                        out=out_arr, 
                                        transform=out.transform,
                                        all_touched=all_touched)
            
            burned = np.where(burned == 1, 1, 0).astype(np.float64)
            
            # apply land mask
            burned *= land_mask
            
            # make nan excluded
            burned = np.where(np.isnan(burned), 1, burned)

            out.write_band(1, burned.astype(np.int16))


### 3.2 Set standard raster template


In [11]:
# open the template raster and extract metadata and land mask
with rasterio.open(template_raster) as template:

    metadata = template.meta.copy()

    # update raster data type
    metadata.update(dtype=np.float64)

    # extract land mask
    land_mask = template.read(1)
    land_mask = np.where(land_mask == 0, np.nan, 1)

    # get the template CRS
    template_crs = template.crs


### 3.3 Process suitability layers

#### gridcerf_epa_nonattainment_co_conus.tif


- **Title**:  U.S. Environmental Protection Agency (EPA) Carbon Monoxide (1971 Standard)
- **Description from Source**: Boundary shapes for all of the areas that were designated nonattainment for a standard revision
- **Source URL**:  https://www.sciencebase.gov/catalog/item/5748a4cbe4b07e28b664dd79
- **Date Accessed**:  8/1/23
- **Citation**
> Environmental Protection Agency. Green Book GIS Download. https://www.epa.gov/green-book/green-book-gis-download (2023).


In [12]:
# read in shapefile
gdf = gpd.read_file(co_file)

# reproject to GRIDCERF 
gdf = gdf.to_crs(template_crs)

# assign raster value field
gdf["rval"] = 1

# rasterize 
vector_to_raster(template_raster, 
                 gdf, 
                 value_field="rval", 
                 output_raster=output_co_file,
                 all_touched=True)


#### gridcerf_epa_nonattainment_no2_conus.tif

- **Title**:  EPA Nitrogen Dioxide (1971 Standard)
- **Description from Source**: Boundary shapes for all of the areas that were designated nonattainment for a standard revision
- **Source URL**:  https://www.sciencebase.gov/catalog/item/5748a4cbe4b07e28b664dd80
- **Date Accessed**:  8/1/23
- **Citation**
> Environmental Protection Agency. Green Book GIS Download. https://www.epa.gov/green-book/green-book-gis-download (2023).


In [13]:
# read in shapefile
gdf = gpd.read_file(no2_file)

# reproject to GRIDCERF 
gdf = gdf.to_crs(template_crs)

# assign raster value field
gdf["rval"] = 1

# rasterize 
vector_to_raster(template_raster, 
                 gdf, 
                 value_field="rval", 
                 output_raster=output_no2_file,
                 all_touched=True)


#### gridcerf_epa_nonattainment_ozone_conus.tif

- **Title**:  EPA 8-Hour Ozone (2015 Standard)
- **Description from Source**: Boundary shapes for all of the areas that were designated nonattainment for a standard revision
- **Source URL**:  hhttps://www.sciencebase.gov/catalog/item/5748a4cbe4b07e28b664dd81
- **Date Accessed**:  8/1/23
- **Citation**
> Environmental Protection Agency. Green Book GIS Download. https://www.epa.gov/green-book/green-book-gis-download (2023).


In [14]:
# read in shapefile
gdf = gpd.read_file(ozone_file)

# reproject to GRIDCERF 
gdf = gdf.to_crs(template_crs)

# assign raster value field
gdf["rval"] = 1

# rasterize 
vector_to_raster(template_raster, 
                 gdf, 
                 value_field="rval", 
                 output_raster=output_ozone_file,
                 all_touched=True)


#### gridcerf_epa_nonattainment_lead_conus.tif

- **Title**:  EPA Lead (2008 Standard)
- **Description from Source**: Boundary shapes for all of the areas that were designated nonattainment for a standard revision
- **Source URL**:  https://www.sciencebase.gov/catalog/item/5748a4cbe4b07e28b664dd82
- **Date Accessed**:  8/1/23
- **Citation**
> Environmental Protection Agency. Green Book GIS Download. https://www.epa.gov/green-book/green-book-gis-download (2023).


In [15]:
# read in shapefile
gdf = gpd.read_file(lead_file)

# reproject to GRIDCERF 
gdf = gdf.to_crs(template_crs)

# assign raster value field
gdf["rval"] = 1

# rasterize 
vector_to_raster(template_raster, 
                 gdf, 
                 value_field="rval", 
                 output_raster=output_lead_file,
                 all_touched=True)


#### gridcerf_epa_nonattainment_pm10_conus.tif

- **Title**:  EPA PM-10 (1987 Standard)
- **Description from Source**: Boundary shapes for all of the areas that were designated nonattainment for a standard revision
- **Source URL**:  https://www.sciencebase.gov/catalog/item/5748a4cbe4b07e28b664dd83
- **Date Accessed**:  8/1/23
- **Citation**
> Environmental Protection Agency. Green Book GIS Download. https://www.epa.gov/green-book/green-book-gis-download (2023).


In [16]:
# read in shapefile
gdf = gpd.read_file(pm10_file)

# reproject to GRIDCERF 
gdf = gdf.to_crs(template_crs)

# assign raster value field
gdf["rval"] = 1

# rasterize 
vector_to_raster(template_raster, 
                 gdf, 
                 value_field="rval", 
                 output_raster=output_pm10_file,
                 all_touched=True)


#### gridcerf_epa_nonattainment_pm2p5_conus.tif

- **Title**:  EPA PM-2.5 (2012 Standard)
- **Description from Source**: Boundary shapes for all of the areas that were designated nonattainment for a standard revision
- **Source URL**:  https://www.sciencebase.gov/catalog/item/5748a4cbe4b07e28b664dd84
- **Date Accessed**:  8/1/23
- **Citation**
> Environmental Protection Agency. Green Book GIS Download. https://www.epa.gov/green-book/green-book-gis-download (2023).


In [17]:
# read in shapefile
gdf = gpd.read_file(pm25_file)

# reproject to GRIDCERF 
gdf = gdf.to_crs(template_crs)

# assign raster value field
gdf["rval"] = 1

# rasterize 
vector_to_raster(template_raster, 
                 gdf, 
                 value_field="rval", 
                 output_raster=output_pm25_file,
                 all_touched=True)


#### gridcerf_epa_nonattainment_so2_conus.tif


- **Title**:  EPA Sulfur Dioxide (2010 Standard)
- **Description from Source**: Boundary shapes for all of the areas that were designated nonattainment for a standard revision
- **Source URL**:  https://www.sciencebase.gov/catalog/item/5748a4cbe4b07e28b664dd85
- **Date Accessed**:  8/1/23
- **Citation**
> Environmental Protection Agency. Green Book GIS Download. https://www.epa.gov/green-book/green-book-gis-download (2023).


In [18]:
# read in shapefile
gdf = gpd.read_file(so2_file)

# reproject to GRIDCERF 
gdf = gdf.to_crs(template_crs)

# assign raster value field
gdf["rval"] = 1

# rasterize 
vector_to_raster(template_raster, 
                 gdf, 
                 value_field="rval", 
                 output_raster=output_so2_file,
                 all_touched=True)
